# 1225. Report Contiguous Dates
[問題](https://leetcode.com/problems/report-contiguous-dates/description/?envType=study-plan-v2&envId=premium-sql-50)

## 解答の方針
連続性の抽出なので、gaps and islandsを使う。  



In [1]:
import pandas as pd

In [145]:
data = [['2018-12-28'], ['2018-12-29'], ['2019-01-04'], ['2019-01-05']]
failed = pd.DataFrame(data, columns=['fail_date']).astype({'fail_date':'datetime64[ns]'})
data = [['2018-12-30'], ['2018-12-31'], ['2019-01-01'], ['2019-01-02'], ['2019-01-03'], ['2019-01-06']]
succeeded = pd.DataFrame(data, columns=['success_date']).astype({'success_date':'datetime64[ns]'})

In [146]:
failed

,fail_date
0,2018-12-28
1,2018-12-29
2,2019-01-04
3,2019-01-05


In [147]:
succeeded

,success_date
0,2018-12-30
1,2018-12-31
2,2019-01-01
3,2019-01-02
4,2019-01-03
5,2019-01-06


In [148]:
def report_contiguous_dates(failed: pd.DataFrame, succeeded: pd.DataFrame) -> pd.DataFrame:
    failed["state"] = "failed"
    succeeded["state"] = "succeeded"
    failed = failed.rename({"fail_date": "date"}, axis=1)
    succeeded = succeeded.rename({"success_date": "date"}, axis=1)
    df = pd.concat([failed, succeeded])
    df = df[(df["date"] >= '2019-01-01') & (df["date"] < '2020-01-01')]
    df = df.sort_values("date").reset_index(drop=True)
    df["next"] = df["state"].shift()
    df["period"] = (df["state"] != df["next"]).cumsum()
    result = df.groupby(["state", "period"]).agg(start_date=('date', 'min'), end_date=('date', 'max')).reset_index()
    result = result.rename({"state": "period_state"}, axis=1)
    result = result[["period_state", "start_date", "end_date"]].sort_values("start_date")
    return result


In [149]:
report_contiguous_dates(failed, succeeded)

,period_state,start_date,end_date
1,succeeded,2019-01-01,2019-01-03
0,failed,2019-01-04,2019-01-05
2,succeeded,2019-01-06,2019-01-06


## Test scripts

In [93]:
failed["state"] = "failed"
succeeded["state"] = "succeeded"

In [97]:
failed = failed.rename({"fail_date": "date"}, axis=1)
succeeded = succeeded.rename({"success_date": "date"}, axis=1)

In [129]:
df = pd.concat([failed, succeeded])

In [130]:
df

,date,state
0,2018-12-28,failed
1,2018-12-29,failed
2,2019-01-04,failed
3,2019-01-05,failed
0,2018-12-30,succeeded
1,2018-12-31,succeeded
2,2019-01-01,succeeded
3,2019-01-02,succeeded
4,2019-01-03,succeeded
5,2019-01-06,succeeded


In [131]:
df = df[(df["date"] >= '2019-01-01') & (df["date"] < '2020-01-01')]

In [132]:
df = df.sort_values("date").reset_index(drop=True)

In [133]:
df["next"] = df["state"].shift()

In [134]:
df["period"] = (df["state"] != df["next"]).cumsum()

In [135]:
df

,date,state,next,period
0,2019-01-01,succeeded,None,1
1,2019-01-02,succeeded,succeeded,1
2,2019-01-03,succeeded,succeeded,1
3,2019-01-04,failed,succeeded,2
4,2019-01-05,failed,failed,2
5,2019-01-06,succeeded,failed,3


In [139]:
result = df.groupby(["state", "period"]).agg(start_date=("date", "min"), end_date=("date", "max")).reset_index()

In [141]:
result = result.rename({"state": "period_state"},axis=1)

In [143]:
result[["period_state", "start_date", "end_date"]].sort_values("start_date")

,period_state,start_date,end_date
1,succeeded,2019-01-01,2019-01-03
0,failed,2019-01-04,2019-01-05
2,succeeded,2019-01-06,2019-01-06
